In [ ]:
!pip uninstall -y pydantic
!pip install pydantic==1.10.7
!pip install langchain --upgrade
!pip install langchain_community
!pip install openai
!pip install gradio


In [ ]:
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.schema import ChatMessage

import json

In [ ]:
# Função para carregar as informações da pizzaria de um arquivo JSON
def carregar_informacoes_pizzaria():
    with open('config_pizzaria.json', 'r', encoding='utf-8') as file:
        return json.load(file)

def carregar_template_contexto():
    with open('context_template.txt', 'r', encoding='utf-8') as file:
        return file.read()

In [ ]:
# Função para carregar o contexto com informações dinâmicas
def getContexto():
    info = carregar_informacoes_pizzaria()
    template_contexto = carregar_template_contexto()
    cardapio_str = ', '.join(info['cardapio'].keys())

    contexto = template_contexto.format(
        nome=info['nome'],
        endereco=info['endereco'],
        horario_funcionamento=info['horario_funcionamento'],
        telefone=info['telefone'],
        email=info['email'],
        site=info['site'],
        cardapio=cardapio_str,
        promocoes=info['promocoes'],
        forma_pagamento=", ".join(info['forma_pagamento']),
        taxa_entrega=info['taxa_entrega']
    )

    return contexto


In [ ]:
#Função para inicializar o histórico de mensagens
def start_chat(openai_api_key):
    if not openai_api_key:
        return [], "Por favor, insira sua chave da API."

    # Inicializa as mensagens com uma mensagem do assistente
    messages = [("assistant", "Fala, beleza? Como posso te ajudar? Pergunte sobre nossa pizzaria!")]
    return messages, ""


In [ ]:
contexto = getContexto()


In [ ]:
# Função para gerar a resposta do chatbot com contexto da pizzaria
def chatbot_response(messages, user_input, openai_api_key, model="gpt-3.5-turbo", temperature=0.3, max_tokens=250):
    try:
        if not openai_api_key:
            return messages, "", "Por favor, insira a chave da API."

        # Inicializa o modelo de linguagem com a chave da API da OpenAI e parâmetros personalizados
        llm = ChatOpenAI(
            openai_api_key=openai_api_key,
            model=model,          # Define o modelo
            temperature=temperature,  # Define a temperatura
            max_tokens=max_tokens     # Define o número máximo de tokens
        )

        # Adiciona a mensagem do usuário ao histórico
        messages.append(("user", user_input))

       # Converte as mensagens para o formato esperado e adiciona o contexto
        conversation = [ChatMessage(role="system", content=contexto)] + [ChatMessage(role=msg[0], content=msg[1]) for msg in messages]

        # Gera a resposta
        response = llm(conversation)

        # Adiciona a resposta do assistente ao histórico
        messages.append(("assistant", response.content))

        return messages, "", ""  # Retorna as mensagens atualizadas e limpa o input
    except Exception as e:
        return messages, "", f"Erro: {str(e)}"



In [ ]:
# Função para inicializar o histórico de mensagens
def start_chat(openai_api_key):
    if not openai_api_key:
        return [], "Por favor, insira sua chave da API."

    # Inicializa as mensagens com uma mensagem do assistente
    messages = [("assistant", "Seja bem-vindo à Pizzaria BOT. Como posso te ajudar?")]
    return messages, ""



In [ ]:

# Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("# Chatbot da Pizzaria BOT usando OpenAI API e Gradio")

    # Campo para inserção da chave da API da OpenAI
    openai_api_key = gr.Textbox(label="Chave da API da OpenAI", type="password")

    # Caixa de chat exibindo as mensagens
    chatbox = gr.Chatbot(label="Chatbot")

    # Entrada do usuário
    user_input = gr.Textbox(label="Sua mensagem", interactive=False)

    # Entrada para o modelo
    model_input = gr.Textbox(label="Modelo OpenAI", value="gpt-3.5-turbo", interactive=True)

    # Entrada para a temperatura
    temperature_input = gr.Slider(label="Temperatura", minimum=0, maximum=1, value=0.3)

    # Entrada para o número máximo de tokens
    max_tokens_input = gr.Number(label="Máximo de Tokens", value=250)

    # Mensagem de erro (se houver)
    error_output = gr.Markdown()

    # Botão para iniciar a conversa
    btn_start = gr.Button("Iniciar Chat")

    # Função de clique do botão "Iniciar Chat"
    btn_start.click(
        start_chat,
        inputs=openai_api_key,
        outputs=[chatbox, error_output],
        show_progress=True
    )

    # Atualiza a caixa de chat quando o usuário envia uma nova mensagem
    user_input.submit(
        chatbot_response,
        inputs=[chatbox, user_input, openai_api_key, model_input, temperature_input, max_tokens_input],
        outputs=[chatbox, user_input, error_output],
        show_progress=True
    )

    # Ativa o campo de entrada de texto quando a chave da API for fornecida
    openai_api_key.change(
        lambda key: gr.update(interactive=bool(key)),
        inputs=openai_api_key,
        outputs=user_input
    )

# Lança a interface
demo.launch()
